# CIFAR-10 w/ default parameters

Customization and further experiements will be presented in other notebooks. 

In this notebook, we follow this [tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) provided by PyTorch on the CIFAR-10 dataset as well other online tutorials to understand each individual component. 

---

### Code



- We start by importing the libraries we need to load the CIFAR-10 and build the model we are going to train.

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

- In order to work with the images, we define a *transforms.Compose* that will be used to place them in a *Tensor* and then normalize each channel (RGB) such that values are in the [-1, 1] range instead of the [0, 1]

In [0]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                                     std=(0.5, 0.5, 0.5))])

- Now, we can load the CIFAR-10 data into train and test sets while transforming the images with the transformer defined above. We use the PyTorch Dataload to automatically split the two sets into smaller batches (iterating through them later will be faster). 

In [0]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


- In order to improve speed of this task, training and testing will be done using the GPU provided by Colab instead of the default CPU.

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


-  We are now ready to create our model for classifying the images. We construct the model by superposing successively the following hidden layers and using ReLU as the activation layer for every layer:
    1.   A convolutional layer with **3 input channels** (R, G, and B) and **6 output channels** using **5x5 filters/kernels** 
    2.   A max pooling layer with a **stride of 2**
    3.   A convolutional layer with **6 input channels** and **16 output channels** using **5x5 filters** 
    4.   A linear layer with **400 inputs** (16 channels of size 5x5) and **120 outputs**
    5.   A linear layer with **120 inputs** and **84 outputs**
    6.   A linear layer with **84 inputs** and **10 outputs** corresponding to the 10 classes we have in this classification task. 

In [0]:
import torch.nn as nn
import torch.nn.functional as func


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """
        Defines a forward pass of the data through the model.
        Connects the different layers in the proper order. 
        Uses ReLU as the activation function of the hidden units. 
        """
        x = self.pool(func.relu(self.conv1(x)))
        x = self.pool(func.relu(self.conv2(x)))
        # Reshape the tensor variable so it has 400 columns 
        # Let PyTorch figure out how many rows are needed (-1)
        x = x.view(-1, 400)
        x = func.relu(self.fc1(x))
        x = func.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate our model to train it and evaluate it
net = ConvNet()
# Use the GPU for operations on the net
net.to(device)

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

- After defining our model, we choose the cross entropy loss as our loss function and use stochastic gradient descent with momentum to optimize our model.

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

- Now train the model

In [0]:
for epoch in range(11):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs and use them with the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # clear the gradients computed in the previous pass
        optimizer.zero_grad()

        # forward pass + compare to true labels
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        # Backprop: calculate the gradient for every parameter x and add to x.grad
        loss.backward()
        
        # Subtract the gradient from current value
        # while taking into account momentum and learning rate
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.237
[1,  4000] loss: 1.894
[1,  6000] loss: 1.685
[1,  8000] loss: 1.586
[1, 10000] loss: 1.503
[1, 12000] loss: 1.459
[2,  2000] loss: 1.382
[2,  4000] loss: 1.355
[2,  6000] loss: 1.338
[2,  8000] loss: 1.360
[2, 10000] loss: 1.294
[2, 12000] loss: 1.276
[3,  2000] loss: 1.232
[3,  4000] loss: 1.206
[3,  6000] loss: 1.203
[3,  8000] loss: 1.187
[3, 10000] loss: 1.186
[3, 12000] loss: 1.162
[4,  2000] loss: 1.095
[4,  4000] loss: 1.112
[4,  6000] loss: 1.124
[4,  8000] loss: 1.112
[4, 10000] loss: 1.118
[4, 12000] loss: 1.085
[5,  2000] loss: 1.026
[5,  4000] loss: 1.034
[5,  6000] loss: 1.013
[5,  8000] loss: 1.051
[5, 10000] loss: 1.034
[5, 12000] loss: 1.044
[6,  2000] loss: 0.949
[6,  4000] loss: 0.964
[6,  6000] loss: 0.995
[6,  8000] loss: 0.979
[6, 10000] loss: 0.958
[6, 12000] loss: 1.006
[7,  2000] loss: 0.888
[7,  4000] loss: 0.921
[7,  6000] loss: 0.944
[7,  8000] loss: 0.921
[7, 10000] loss: 0.942
[7, 12000] loss: 0.951
[8,  2000] loss: 0.855
[8,  4000] 

- We evaluate the model on the test set

In [0]:
correct = 0
total = 0
with torch.no_grad(): # do not change the gradients of the variables
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


- Accuracy on each class

In [0]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 64 %
Accuracy of   car : 71 %
Accuracy of  bird : 47 %
Accuracy of   cat : 37 %
Accuracy of  deer : 63 %
Accuracy of   dog : 58 %
Accuracy of  frog : 64 %
Accuracy of horse : 61 %
Accuracy of  ship : 81 %
Accuracy of truck : 75 %
